In [ ]:
import numpy as np
from utils2 import (
    load_kspace_masks, get_lesion_kernel, generate_single_patch_pair
)
from tqdm import tqdm
from pathlib import Path

# Load masks (only once)
load_kspace_masks()

# =============================================
# 1. Configuration for Experiments
# =============================================
experiments = {
    #'exp1': {'amplitude': 0.05, 'noise_levels': [1e-3, 4e-4, 1e-4], 'acceleration': [2, 4]},
    #'exp2': {'amplitude': 0.2, 'noise_levels': [1e-3, 4e-4, 1e-4], 'acceleration': [2, 4]},
    #'exp3': {'amplitude': 0.05, 'noise_levels': [8e-3], 'acceleration': [2, 4]},
    #'exp4': {'amplitude': 0.2, 'noise_levels': [8e-3], 'acceleration': [2, 4]}
    'exp5': {'amplitude': 0.05, 'noise_levels': [4e-3], 'acceleration': [2, 4]},
    'exp6': {'amplitude': 0.2, 'noise_levels': [4e-3], 'acceleration': [2, 4]}
}

# =============================================
# 2. Load MRI Data
# =============================================
MRI_imgs = np.load("C:/Users/garci/FinalReportPark/Data/MRI_imgs/HCP_selected_ones2.npy")
MRI_seg = np.load("C:/Users/garci/FinalReportPark/Data/MRI_seg/HCP_selected_ones2_wmparc.npy")
output_dir = Path("C:/Users/garci/FinalReportPark/Ready_to_train")
output_dir.mkdir(exist_ok=True)

# =============================================
# 3. Patch Generation and Saving Function
# =============================================
def generate_and_save_patches(amplitude, noise_levels, accelerations, max_images=11000):
    for noise in noise_levels:
        for accel in accelerations:
            print(f"\nGenerating patches - intensity: {amplitude}, Noise: {noise}, Acceleration: {accel}x")

            kernel = get_lesion_kernel(rot=30, intensity = amplitude)

            signal_patches = []
            nonsignal_patches = []
            lesion_centers = []
            reference_patches = []

            for i in tqdm(range(max_images), desc="Generating patch pairs"):
                try:
                    sig, nonsig, info, reference = generate_single_patch_pair(
                        MRI_imgs, MRI_seg, i=i, kernel=kernel,
                        noise=noise, aceleration=accel
                    )
                    if sig is not None:
                        signal_patches.append(sig)
                        nonsignal_patches.append(nonsig)
                        lesion_centers.append(info)
                        reference_patches.append(reference)

                except ValueError as e:
                    print(f"Skipped index {i}: {e}")
                    continue

            print(f"Generated {len(signal_patches)} patch pairs.")

            patch_data = {
                'signal_patches': np.array(signal_patches),
                'nonsignal_patches': np.array(nonsignal_patches),
                'lesion_centers': np.array(lesion_centers),
                'reference_patches': np.array(reference_patches),
            }

            filename = f"patches_amp_{amplitude}_noise_{noise}_accel_{accel}.npy"
            np.save(output_dir / filename, patch_data, allow_pickle=True)
            print(f"Saved {filename}")

# =============================================
# 4. Run All Experiments
# =============================================
for exp_name, config in experiments.items():
    print(f"\nStarting experiment: {exp_name}")
    generate_and_save_patches(
        amplitude=config['amplitude'],
        noise_levels=config['noise_levels'],
        accelerations=config['acceleration']
    )

print("\nAll patches generated and saved in separate .npy files!")
print("You can load any file with:")
print("img = np.load('patches_amp_0.05_noise_0.004_accel_4.npy', allow_pickle=True).item()")
print("Then access: signal_patches = img['signal_patches'], etc.")